In [5]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


In [6]:
# Set the URL for the CUNY academic calendar
url = "https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")


In [7]:
# Extract table data
rows = soup.find_all("tr")
data = []

#loop through the rows and extract the data
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 3:
        date = cells[0].text.strip()
        text = cells[2].text.replace("\n", " ").replace("\t", " ").strip()

        # Handle the date ranges
        if "-" in date:
            # Split the date range into start and end dates
            start_date_str, end_date_str = date.split("-")
            start_date_str = f"{start_date_str.strip()}, 2021" 
            month = start_date_str.split()[0]
            end_date_str = f"{month} {end_date_str.strip()}, 2021" 
            # Convert the date strings to datetime objects
            start_date = datetime.strptime(start_date_str, "%B %d, %Y")
            end_date = datetime.strptime(end_date_str, "%B %d, %Y")

            # Generate all of the dates in the range and append to data
            date_range = pd.date_range(start=start_date, end=end_date)
            for single_date in date_range:
                day_of_week = single_date.strftime('%A')
                data.append({"Date": single_date, "Day of Week": day_of_week, "Text": text})
        else:
            # Handle single dates
            if "2022" not in date:
                date = f"{date}, 2021"
            date = date.strip()  
            date_obj = datetime.strptime(date, "%B %d, %Y")
            day_of_week = date_obj.strftime('%A')
            data.append({"Date": date_obj, "Day of Week": day_of_week, "Text": text})